In [1]:
# Blake Myers and Nicolas Stencel, Part 1 of the final project for CSCE 5210
# Creating an extractive text summarizer

In [2]:
import stanza
import csv
import os
import networkx as nx
import vis

In [3]:
class NLP :
  def __init__(self,lang='en'):
    #stanza.download(lang)
    self.nlp = stanza.Pipeline(lang=lang,logging_level='WARN')

  def from_file(self,fname='texts/english'):
    self.fname=fname
    text = file2text(fname + ".txt")
    self.doc = self.nlp(text)

  def from_text(self,text="Hello!"):
    self.doc = self.nlp(text)

  def keynoun(self,x):
    return  x.upos == 'NOUN' and ('subj' in x.deprel or 'ob' in x.deprel)

  def facts(self):
    def fact(x,sent,sid) :
      if x.head==0 :
        yield x.lemma,x.upos+'_PREDICATE_OF',sid,sid
      else :
        hw=sent.words[x.head-1]
        if self.keynoun(x):
          yield hw.lemma, hw.upos + "rev_"+x.deprel + x.upos, x.lemma, sid
          yield (sid, 'ABOUT', x.lemma, sid)
        else:
          yield x.lemma,x.upos+x.deprel+hw.upos,hw.lemma,sid
        if  x.deprel in ("compound","flat") :
          comp = x.lemma+" "+hw.lemma
          yield x.lemma, x.upos+"inCOMPOUND", comp, sid
          yield hw.lemma, hw.upos + "inCOMPOUND", comp, sid
          yield (sid, 'ABOUT', comp, sid)

    for sid,sent in enumerate(self.doc.sentences) :
      for x in sent.words :
        yield from fact(x,sent,sid)

  def keynouns(self):
    '''collects important nouns'''
    ns=set()
    for sent in self.doc.sentences:
      for x in sent.words:
        if self.keynoun(x) :
          ns.add(x.lemma)
    return ns

  def info(self,wk=8,sk=6):
    g=self.to_nx()
    ranks=nx.pagerank(g)
    ns=self.keynouns()
    kwds,sids=ranks2info(ranks,ns,wk,sk)
    sents=list(map(self.get_sent,sorted(sids)))
    return kwds,sents

  def to_nx(self):
    return facts2nx(self.facts())

  def to_tsv(self):
    facts2tsv(self.facts(),"out/"+self.fname+".tsv")
    self.to_sents()

  def to_prolog(self):
    facts2prolog(self.facts(),"out/"+self.fname+".pro")

  def get_sent(self,sid) :
    return self.doc.sentences[sid].text

  def to_sents(self):
    def sent_gen():
       for sid,sent in enumerate(self.doc.sentences):
         yield sid,sent.text
    facts2tsv(sent_gen(),"out/"+self.fname+"_sents.tsv")

  def summarize(self,wk=8,sk=5):
    kws,sents=self.info(wk,sk)
    print("\nSUMMARY:")
    for sent in sents : print(sent)
    print("\nKEYWORDS:")
    for w in kws : print(w,end='; ')
    print("\n")

In [4]:
def file2text(fname) :
  with open(fname,'r') as f:
    return f.read()

def facts2nx(fgen) :
   g=nx.DiGraph()
   for f,rel,t,id in fgen :
     g.add_edge(f,t)
   return g

def ranks2info(ranks,keyns,wk,sk) :
  ranked=sorted(ranks.items(),key=(lambda x: x[1]),reverse=True)
  sids=[]
  kwds=[]
  for x, r in ranked:
    if wk<=0 : break
    if isinstance(x,str) and x in keyns:
      kwds.append(x)
      wk-=1
  for x,r in ranked:
    if sk <= 0: break
    if isinstance(x, int):
      sids.append(x)
      sk -= 1
  return kwds,sids

def facts2tsv(fgen,fname) :
  ensure_path(fname)
  with open(fname, 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    for fact in fgen:
      writer.writerow(fact)

def facts2prolog(fgen,fname) :
  ensure_path(fname)
  with open(fname, 'w') as f:
    for fact in fgen:
      print('edge',end='',file=f)
      print(fact,end=".\n",file=f)

def exists_file(fname):
  return os.path.exists(fname)

def home_dir() :
  from pathlib import Path
  return str(Path.home())

def ensure_path(fname) :
  dir,_=os.path.split(fname)
  os.makedirs(dir, exist_ok=True)

In [5]:
def test(fname='texts/english',lang='en') :
  nlp=NLP(lang)
  nlp.from_file(fname)
  nlp.to_tsv()
  nlp.to_prolog()
  nlp.summarize()

if __name__=="__main__" :
  test(fname='texts/english',lang='en')


SUMMARY:
And the time?
On the stroke of midnight, as a matter of fact.
And, outside the window, fireworks and crowds.
I, Saleem Sinai, later variously called Snotnose, Stainface, Baldy, Sniffer, Buddha and even Piece-of-the-Moon, had become heavily embroiled in Fate—at the best of times a dangerous sort of involvement.
I must work fast, faster than Scheherazade, if I am to end up meaning — yes, meaning — something.

KEYWORDS:
night; time; meaning; instant; year; say; date; thanks; 



In [6]:
f = open('/Users/blakemyers/texts/english.txt', 'r')
file_contents = f.read()
print ("Sample Text: " + file_contents)
f.close()

# The original text that we ran with the above program:

Sample Text: I was born in the city of Bombay … once upon a time. No, that won’t do, there’s no getting away from the date: I was born in Doctor Narlikar’s Nursing Home on August 15th, 1947. And the time? The time matters, too. Well then: at night. No, it’s important to be more … On the stroke of midnight, as a matter of fact. Clock-hands joined palms in respectful greeting as I came. Oh, spell it out, spell it out: at the precise instant of India’s arrival at independence, I tumbled forth into the world. There were gasps. And, outside the window, fireworks and crowds. A few seconds later, my father broke his big toe; but his accident was a mere trifle when set beside what had befallen me in that benighted moment, because thanks to the occult tyrannies of those blandly saluting clocks I had been mysteriously handcuffed to history, my destinies indissolubly chained to those of my country. For the next three decades, there was to be no escape. Soothsayers had prophesied me, newspapers ce

In [7]:
#The summary that we extracted manually:
#I was born in Bombay on August 15th, 1947, at midnight,
#at the precise instant of India’s independence. I will soon be
#thirty-one years old. I must work fast if I am to end up
#meaning something. I fear absurdity."

#The kewords that we extracted manually:
#born; midnight, India; independence; meaning